# Latest point anomaly detection with the Anomaly Detector API

### Use this Jupyter notebook to start visualizing anomalies as a batch with the Anomaly Detector API in Python.

While you can detect anomalies as a batch, you can also detect the anomaly status of the last data point in the time series. This notebook iteratively sends latest-point anomaly detection requests to the Anomaly Detector API and visualizes the response. The graph created at the end of this notebook will display the following:
* Anomalies found while in the data set, highlighted.
* Anomaly detection boundaries 
* Anomalies seen in the data, highlighted.

By calling the API on your data's latest points, you can monitor your data as it's created. 

The following example simulates using the Anomaly Detector API on streaming data. Sections of the example time series are sent to the API over multiple iterations, and the anomaly status of each section's last data point is saved. The data set used in this example has a pattern that repeats roughly every 7 data points (the `period` in the request's JSON file), so for best results, the data set is sent in groups of 29 points (`4 * <period> + an extra data point`. See [Best practices for using the Anomaly Detector API](https://docs.microsoft.com/azure/cognitive-services/anomaly-detector/concepts/anomaly-detection-best-practices) for more information).  

In [1]:
# To start sending requests to the Anomaly Detector API, paste your subscription key below.
subscription_key = 'A COMPLETER' 

# Use the endpoint your received from overview section of the Anomaly Detector resource you created
# the endpoint is like https://westus2.api.cognitive.microsoft.com/, different by regions, you need to concat anomalydetector/v1.0/timeseries/last/detect

endpoint_latest = 'https://westeurope.api.cognitive.microsoft.com/anomalydetector/v1.0/timeseries/last/detect'

In [2]:
import requests
import json
import pandas as pd
import numpy as np
from __future__ import print_function
import warnings
warnings.filterwarnings('ignore')

# Import library to display results
import matplotlib.pyplot as plt
%matplotlib inline 

In [3]:
from bokeh.plotting import figure,output_notebook, show
from bokeh.palettes import Blues4
from bokeh.models import ColumnDataSource,Slider
import datetime
from bokeh.io import push_notebook
from dateutil import parser
from ipywidgets import interact, widgets, fixed
output_notebook()

Loading BokehJS ...

In [4]:
def detect(endpoint, subscription_key, request_data):
    headers = {'Content-Type': 'application/json', 'Ocp-Apim-Subscription-Key': subscription_key}
    response = requests.post(endpoint, data=json.dumps(request_data), headers=headers)
    if response.status_code == 200:
        return json.loads(response.content.decode("utf-8"))
    else:
        print(response.status_code)
        raise Exception(response.text)

In [5]:
def build_figure(result, sample_data, sensitivity):
    columns = {'expectedValues': result['expectedValues'], 'isAnomaly': result['isAnomaly'], 'isNegativeAnomaly': result['isNegativeAnomaly'],
              'isPositiveAnomaly': result['isPositiveAnomaly'], 'upperMargins': result['upperMargins'], 'lowerMargins': result['lowerMargins']
              , 'value': [x['value'] for x in sample_data['series']], 'timestamp': [parser.parse(x['timestamp']) for x in sample_data['series']]}
    response = pd.DataFrame(data=columns)
    values = response['value']
    label = response['timestamp']
    anomalies = []
    anomaly_labels = []
    index = 0
    anomaly_indexes = []
    p = figure(x_axis_type='datetime', title="Anomaly Detection Result ({0} Sensitivity)".format(sensitivity), width=800, height=600)
    for anom in response['isAnomaly']:
        if anom == True and (values[index] > response.iloc[index]['expectedValues'] + response.iloc[index]['upperMargins'] or 
                         values[index] < response.iloc[index]['expectedValues'] - response.iloc[index]['lowerMargins']):
            anomalies.append(values[index])
            anomaly_labels.append(label[index])
            anomaly_indexes.append(index)
        index = index+1
    upperband = response['expectedValues'] + response['upperMargins']
    lowerband = response['expectedValues'] -response['lowerMargins']
    band_x = np.append(label, label[::-1])
    band_y = np.append(lowerband, upperband[::-1])
    boundary = p.patch(band_x, band_y, color=Blues4[2], fill_alpha=0.5, line_width=1, legend='Boundary')
    p.line(label, values, legend='value', color="#2222aa", line_width=1)
    p.line(label, response['expectedValues'], legend='expectedValue',  line_width=1, line_dash="dotdash", line_color='olivedrab')
    anom_source = ColumnDataSource(dict(x=anomaly_labels, y=anomalies))
    anoms = p.circle('x', 'y', size=5, color='tomato', source=anom_source)
    p.legend.border_line_width = 1
    p.legend.background_fill_alpha  = 0.1
    show(p, notebook_handle=True)

### Detect latest anomaly of sample timeseries

The following cells call the Anomaly Detector API with an example time series data set and different sensitivities for anomaly detection. Varying the sensitivity of the Anomaly Detector API can improve how well the response fits your data.  

In [6]:
def detect_anomaly(sensitivity):
    sample_data = json.load(open('sample.json'))
    points = sample_data['series']
    skip_point = 29
    result = {'expectedValues': [None]*len(points), 'upperMargins': [None]*len(points), 
              'lowerMargins': [None]*len(points), 'isNegativeAnomaly': [False]*len(points), 
              'isPositiveAnomaly':[False]*len(points), 'isAnomaly': [False]*len(points)}
    anom_count = 0
    for i in range(skip_point, len(points)+1):
        single_sample_data = {}
        single_sample_data['series'] = points[i-29:i]
        single_sample_data['granularity'] = 'daily'
        single_sample_data['maxAnomalyRatio'] = 0.25
        single_sample_data['sensitivity'] = sensitivity
        single_point = detect(endpoint_latest, subscription_key, single_sample_data)
        if single_point['isAnomaly'] == True:
            anom_count = anom_count + 1

        result['expectedValues'][i-1] = single_point['expectedValue']
        result['upperMargins'][i-1] = single_point['upperMargin']
        result['lowerMargins'][i-1] = single_point['lowerMargin']
        result['isNegativeAnomaly'][i-1] = single_point['isNegativeAnomaly']
        result['isPositiveAnomaly'][i-1] = single_point['isPositiveAnomaly']
        result['isAnomaly'][i-1] = single_point['isAnomaly']
    
    build_figure(result, sample_data, sensitivity)

In [7]:
# 95 sensitvity
detect_anomaly(95)

In [8]:
# 85 sensitvity
detect_anomaly(85)